# Prompt Engineering Lifecycle

### Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()  

import os
print("LangSmith Key Set:", os.getenv("LANGCHAIN_API_KEY") is not None)


LangSmith Key Set: True


### Log a trace

In [2]:
from app import langsmith_rag

question = "How do I set up tracing to LangSmith with @traceable?"
langsmith_rag(question)

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\DELL\Desktop\Jupyter\mat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching pages: 100%|#############################################################################| 197/197 [00:32<00:00,  6.07it/s]


"To set up tracing to LangSmith with @traceable, you need to set the LANGSMITH_TRACING environment variable to 'true' and the LANGSMITH_API_KEY environment variable to your API key. You can then use the @traceable decorator in Python to log traces to LangSmith. Simply decorate any function with @traceable to log traces, and use the await keyword when calling wrapped sync functions to ensure correct logging."

### Create a Dataset

Let's create a dataset to evaluate this particular step of our application

In [3]:
from langsmith import Client

example_dataset = [
    (
        "How do I enable tracing in a LangChain application?",
        "LangChain supports native tracing through LangSmith by setting environment variables and wrapping chains or tools with decorators.",
        "To enable tracing in a LangChain app, set the environment variable `LANGSMITH_TRACING=true`, and ensure your `LANGSMITH_API_KEY` is set. Optionally, wrap custom components with the `@traceable` decorator."
    ),
    (
        "What is the difference between LangChain and LangSmith?",
        "LangChain is a framework for building LLM applications. LangSmith is a developer platform for debugging, testing, and monitoring those applications.",
        "LangChain provides abstractions like chains, agents, and tools for LLM apps. LangSmith offers observability, evaluation, and dataset management to monitor and improve them."
    ),
    (
        "How do I log custom LLM calls in LangSmith?",
        "LangSmith provides the `RunTree` class, which allows you to manually construct and submit traces for custom logic or non-standard LLM calls.",
        "You can use `RunTree` to manually create and log a trace:\n\n```python\nfrom langsmith.run_trees import RunTree\n\nrun = RunTree(name='Custom LLM Call', run_type='llm', inputs={'prompt': '...your prompt...'})\n# Run your LLM call here\nrun.end(outputs={'response': '...your output...'})\nrun.postRun()\n```"
    ),
    (
        "Can I evaluate prompt templates in LangSmith?",
        "LangSmith lets you evaluate prompts by linking them to datasets and running chains or LLMs against example inputs.",
        "Yes. You can upload a prompt template to Prompt Hub, hydrate it with inputs from a dataset, and evaluate its performance using built-in or custom evaluators."
    ),
    (
        "How do I create and share prompt templates in LangSmith?",
        "The Prompt Hub lets you create, version, and share prompts with others. Templates can be used in LangChain directly via `load_prompt()`.",
        "To share a prompt, create it in the LangSmith UI under Prompt Hub, then toggle the visibility to public or share the unique prompt URL with your team."
    ),
]

client = Client()
dataset_name = "LangSmith Q&A Updated"


try:
    dataset = client.read_dataset(dataset_name)
    print(f"Dataset '{dataset_name}' already exists. Using the existing one.")
except:
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Updated technical questions about LangSmith and LangChain"
    )

# Prepare inputs and outputs
inputs = [{"question": q, "context": c} for q, c, _ in example_dataset]
outputs = [{"output": o} for _, _, o in example_dataset]

# Create examples
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset.id,
)

print(f" Dataset '{dataset_name}' now has {len(inputs)} examples.")


 Dataset 'LangSmith Q&A Updated' now has 5 examples.


### Update our Application to use Prompt Hub

We're going to pretty much define the same RAG application as before - with one crucial improvement.

Instead of pulling our `RAG_PROMPT` from utils.py, we're going to connect to the Prompt Hub in LangSmith.

Let's add the code snippet that will pull down our prompt that we just iterated on!

In [8]:
from langsmith import Client
client = Client()

dataset = client.read_dataset(dataset_name="LangSmith Q&A Updated")


In [31]:
import os
import tempfile
from typing import List
import nest_asyncio

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langsmith import traceable
from langsmith.client import convert_prompt_to_openai_format
from groq import Groq


MODEL_NAME = "llama-3.3-70b-versatile"  # Groq-supported model
MODEL_PROVIDER = "groq"
APP_VERSION = 1.0

nest_asyncio.apply()
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    from langchain_community.embeddings import HuggingFaceEmbeddings
    embd = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    loader = SitemapLoader(
        web_path="https://docs.smith.langchain.com/sitemap.xml",
        continue_on_failure=True
    )
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    splits = splitter.split_documents(docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="llm", metadata={"ls_provider": MODEL_PROVIDER, "ls_model_name": MODEL_NAME})
def call_groq(messages: List[dict]) -> str:
    return groq_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages
    )

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    formatted_prompt = prompt.invoke({
        "language": "English",
        "question": question,
    })

    messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]
    return call_groq(messages)


@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content 


In [32]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("life")  # replace with your prompt name


In [33]:
formatted_prompt = prompt.invoke({
    "language": "Greek",  # or any language you want here
    "question": question,
})


In [ ]:
response = langsmith_rag(question)
print(response)
